In [1]:
import sys, pathlib, os
from dotenv import load_dotenv
repo_root = pathlib.Path.cwd().resolve().parent  # if notebook is in notebooks/
sys.path.append(str(repo_root))
load_dotenv(repo_root / ".env")
print("Repo root:", repo_root)
print("GPU_ENABLED:", os.getenv("GPU_ENABLED"))


Repo root: /home/henry/dev
GPU_ENABLED: true


In [1]:
import sys, pathlib

repo_root = pathlib.Path.cwd()
if not (repo_root / "src").exists():
    # If running from notebooks/ directory, go up one level
    repo_root = repo_root.parent
sys.path.append(str(repo_root))
print("Using repo root:", repo_root)


Using repo root: /home/henry/dev/nitrogen-features


In [2]:
# ensure current working directory is in sys.path
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd()))


sanity checks: imports and config

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from src.datasources.cdl_loader import load_cdl_year, build_stable_corn_mask_from_years


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import box
from src.datasources.cdl_loader import load_cdl_year, build_stable_corn_mask_from_years


In [4]:
import torch
print(torch.cuda.is_available(), torch.cuda.device_count())


True 1


load cdl for multiple years, heavy compute, run this in terminal first


In [5]:
import os, torch
print("GPU_ENABLED:", os.getenv("GPU_ENABLED"))
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device count:", torch.cuda.device_count())
    print("Device 0:", torch.cuda.get_device_name(0))


GPU_ENABLED: true
CUDA available: True
Device count: 1
Device 0: NVIDIA GeForce RTX 4060


In [3]:
years = [2019, 2021, 2023]

cdl_maps = {}
for y in years:
    data, transform, crs = load_cdl_year(y)
    cdl_maps[y] = data


: 

plot corn masks by year

In [4]:
CORN_CLASS = 1  # Confirm this matches your CDL mapping

fig, axes = plt.subplots(1, len(years), figsize=(15, 5))

for i, y in enumerate(years):
    corn_mask = (cdl_maps[y] == CORN_CLASS)
    axes[i].imshow(corn_mask, cmap="gray")
    axes[i].set_title(f"Corn Mask {y}")
    axes[i].axis("off")

plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

build and plot stable corn masks

In [3]:
stable_mask, _, _ = build_stable_corn_mask_from_years(years)

plt.figure(figsize=(6, 6))
plt.imshow(stable_mask, cmap="gray")
plt.title("Stable Corn Mask (All Years)")
plt.axis("off")
plt.show()

print("Stable corn pixel count:", stable_mask.sum())


NameError: name 'build_stable_corn_mask_from_years' is not defined

visually verify AOI

In [2]:
import matplotlib.pyplot as plt
from shapely.geometry import box
from src.geo.aoi_nebraska import NEBRASKA_BBOX

poly = box(*NEBRASKA_BBOX)
x, y = poly.exterior.xy

plt.figure(figsize=(5, 5))
plt.plot(x, y)
plt.title("Nebraska AOI Bounding Box")
plt.axis("equal")
plt.grid()
plt.show()


ModuleNotFoundError: No module named 'matplotlib'